In [119]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import Input
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, SimpleRNN, GRU, Activation
%matplotlib inline

# Считывание данных

In [56]:
id_value = 19
# считывание тренировочных данных
train_df = pd.read_csv('PM_train.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
#train_df=train_df[train_df['id'] == id_value]

In [4]:
# считывание тестовых данных
test_df = pd.read_csv('PM_test.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
test_df=test_df[test_df['id'] == id_value]

In [5]:
# read ground truth data
truth_df = pd.read_csv('PM_truth.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [6]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
3776,19,1,0.0045,-0.0000,100.0,518.67,642.55,1589.99,1411.09,14.62,...,521.86,2388.07,8122.16,8.4148,0.03,392,2388,100.0,38.76,23.2468
3777,19,2,-0.0010,0.0001,100.0,518.67,642.82,1587.77,1411.04,14.62,...,521.31,2388.14,8123.07,8.4086,0.03,392,2388,100.0,38.83,23.2836
3778,19,3,-0.0010,0.0004,100.0,518.67,643.37,1587.33,1407.56,14.62,...,521.74,2388.12,8121.69,8.4316,0.03,392,2388,100.0,38.69,23.2071
3779,19,4,0.0014,0.0002,100.0,518.67,642.53,1590.93,1412.13,14.62,...,521.60,2388.12,8123.21,8.4634,0.03,393,2388,100.0,38.81,23.2752
3780,19,5,-0.0005,-0.0004,100.0,518.67,642.41,1588.13,1407.64,14.62,...,521.21,2388.11,8127.37,8.4570,0.03,393,2388,100.0,38.79,23.2524


# Обработка данных

In [57]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [58]:
# generate label columns for training data
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0,0


In [59]:
# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.01108


In [60]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,19,1,39.005747,333.833333,-100.0,-518.67,-193.093292,-34.240095,-23.333677,-14.62,...,-0.03,-32.333333,-2388.0,-100.0,-28.816828,-30.034866,0.00000,NaN,0,0
1,19,2,30.385057,250.500000,-100.0,-518.67,-193.090566,-34.249919,-23.332141,-14.62,...,-0.03,-32.321429,-2388.0,-100.0,-28.851668,-30.371116,0.00277,NaN,0,0
2,19,3,27.511494,500.500000,-100.0,-518.67,-193.076937,-34.239646,-23.335376,-14.62,...,-0.03,-32.321429,-2388.0,-100.0,-28.703597,-30.245694,0.00554,NaN,0,0
3,19,4,9.120690,333.833333,-100.0,-518.67,-193.146446,-34.245621,-23.330216,-14.62,...,-0.03,-32.309524,-2388.0,-100.0,-28.790698,-30.593292,0.00831,NaN,0,0
4,19,5,32.109195,667.166667,-100.0,-518.67,-193.132816,-34.250460,-23.337976,-14.62,...,-0.03,-32.333333,-2388.0,-100.0,-28.851668,-30.427257,0.01108,NaN,0,0


In [61]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [62]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,19,1,39.005747,333.833333,-100.0,-518.67,-193.093292,-34.240095,-23.333677,-14.62,...,-0.03,-32.333333,-2388.0,-100.0,-28.816828,-30.034866,0.00000,NaN,0,0
1,19,2,30.385057,250.500000,-100.0,-518.67,-193.090566,-34.249919,-23.332141,-14.62,...,-0.03,-32.321429,-2388.0,-100.0,-28.851668,-30.371116,0.00277,NaN,0,0
2,19,3,27.511494,500.500000,-100.0,-518.67,-193.076937,-34.239646,-23.335376,-14.62,...,-0.03,-32.321429,-2388.0,-100.0,-28.703597,-30.245694,0.00554,NaN,0,0
3,19,4,9.120690,333.833333,-100.0,-518.67,-193.146446,-34.245621,-23.330216,-14.62,...,-0.03,-32.309524,-2388.0,-100.0,-28.790698,-30.593292,0.00831,NaN,0,0
4,19,5,32.109195,667.166667,-100.0,-518.67,-193.132816,-34.250460,-23.337976,-14.62,...,-0.03,-32.333333,-2388.0,-100.0,-28.851668,-30.427257,0.01108,NaN,0,0


In [63]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,19,1,39.005747,333.833333,-100.0,-518.67,-193.093292,-34.240095,-23.333677,-14.62,...,-0.03,-32.333333,-2388.0,-100.0,-28.816828,-30.034866,0.00000,NaN,0,0
1,19,2,30.385057,250.500000,-100.0,-518.67,-193.090566,-34.249919,-23.332141,-14.62,...,-0.03,-32.321429,-2388.0,-100.0,-28.851668,-30.371116,0.00277,NaN,0,0
2,19,3,27.511494,500.500000,-100.0,-518.67,-193.076937,-34.239646,-23.335376,-14.62,...,-0.03,-32.321429,-2388.0,-100.0,-28.703597,-30.245694,0.00554,NaN,0,0
3,19,4,9.120690,333.833333,-100.0,-518.67,-193.146446,-34.245621,-23.330216,-14.62,...,-0.03,-32.309524,-2388.0,-100.0,-28.790698,-30.593292,0.00831,NaN,0,0
4,19,5,32.109195,667.166667,-100.0,-518.67,-193.132816,-34.250460,-23.337976,-14.62,...,-0.03,-32.333333,-2388.0,-100.0,-28.851668,-30.427257,0.01108,NaN,0,0


# Моделирование

In [64]:
# pick a large window size of 50 cycles
sequence_length = 50

In [65]:
# preparing data for visualizations 
# window of 50 cycles prior to a failure point for engine id 3
engine_id3 = test_df[test_df['id'] == 3]
engine_id3_50cycleWindow = engine_id3[engine_id3['RUL'] <= engine_id3['RUL'].min() + 50]
cols1 = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10']
engine_id3_50cycleWindow1 = engine_id3_50cycleWindow[cols1]
cols2 = ['s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
engine_id3_50cycleWindow2 = engine_id3_50cycleWindow[cols2]

In [66]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [67]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

In [68]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

In [69]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(15631, 50, 25)

In [70]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [71]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

# LSTM

In [134]:
seq_array.shape

(15631, 50, 25)

In [144]:
sequence_length

50

In [120]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(SimpleRNN(input_shape=(sequence_length, nb_features),units=100,return_sequences=True))
model.add(Dropout(0.2))

model.add(SimpleRNN(units=50,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [142]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(input_shape=(sequence_length, nb_features),units=100,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [143]:
print(model.summary())

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_31 (LSTM)              (None, 50, 100)           50400     
                                                                 
 dropout_46 (Dropout)        (None, 50, 100)           0         
                                                                 
 lstm_32 (LSTM)              (None, 50)                30200     
                                                                 
 dropout_47 (Dropout)        (None, 50)                0         
                                                                 
 dense_26 (Dense)            (None, 1)                 51        
                                                                 
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None


In [145]:
4 * ((50 + 1) * 25 + 25**2)
4 * (50*25 + 100**2 + 100)
#(i×h + h×o) + (h+o)
(50*100 + 100*50) + (100+50)

10150

In [124]:
%%time
# fit the network
history=model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Epoch 1/10
75/75 [==============================] - 5s 71ms/step - loss: 0.0857 - accuracy: 0.9646 - val_loss: 0.0639 - val_accuracy: 0.9731
Epoch 2/10
75/75 [==============================] - 5s 70ms/step - loss: 0.0787 - accuracy: 0.9668 - val_loss: 0.0612 - val_accuracy: 0.9719
Wall time: 10.6 s


In [27]:
history.history

{'loss': [2.3945372104644775, 2.3085525035858154],
 'accuracy': [0.20588235557079315, 0.36274510622024536],
 'val_loss': [9.031469345092773, 9.33469295501709],
 'val_accuracy': [0.0, 0.0]}

In [101]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

79/79 [==============================] - 3s 32ms/step - loss: 1.7724 - accuracy: 0.1983
Accurracy: 0.19832384586334229


In [102]:
# make predictions and compute confusion matrix
y_pred = np.argmax(model.predict(seq_array,verbose=1, batch_size=200),axis=1)
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

79/79 [==============================] - 3s 31ms/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[    0, 12027,   504],
       [    0,  3100,     0],
       [    0,     0,     0]], dtype=int64)

In [127]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  0.0 
 recall =  0.0


D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [105]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(1, 50, 25)

In [106]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

In [107]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

(1, 1)

In [108]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(1, 50, 25)
(1, 1)


In [109]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

1/1 - 0s - loss: 1.9200 - accuracy: 1.0000 - 20ms/epoch - 20ms/step
Accurracy: 1.0


In [114]:
seq_array_test_last.shape

(15631, 50, 25)

In [112]:
model.predict(seq_array_test_last,verbose=1, batch_size=200)

1/1 [==============================] - 0s 20ms/step


array([[ 1.3629744 , -1.0806334 ,  0.5664152 ,  0.2234245 ,  0.3992709 ,
         0.82614815, -1.2616562 , -0.17474172,  0.45274743, -0.230837  ]],
      dtype=float32)

In [111]:
# make predictions and compute confusion matrix
y_pred_test = np.argmax(model.predict(seq_array_test_last),axis=1)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[1]], dtype=int64)

In [40]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

Precision:  0.0 
 Recall:  0.0 
 F1-score: nan


D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\75BD~1\AppData\Local\Temp/ipykernel_22720/2317648545.py:4: RuntimeWarning: invalid value encountered in double_scalars
  f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)


In [41]:
results_df = pd.DataFrame([[scores_test[1],precision_test,recall_test,f1_test],
                          [0.94, 0.952381, 0.8, 0.869565]],
                         columns = ['Accuracy', 'Precision', 'Recall', 'F1-score'],
                         index = ['LSTM',
                                 'Template Best Model'])
results_df

,Accuracy,Precision,Recall,F1-score
LSTM,0.913979,0.000000,0.0,NaN
Template Best Model,0.940000,0.952381,0.8,0.869565
